# *JS-son* - Data Science Demo
This Jupyer notebook provides an example of how *[JS-son](https://github.com/TimKam/JS-son)* can be integrated with state-of-the-art *data science* tools: Jupyter notebooks and Python-based analytics libraries.

## Introduction

In this notebook, we implement a simple information spread simulation, using JS-son's full belief-desire-intention-plan approach and use Python tools to conduct an explorative data analysis of how changes in parameters affect the simulation results. The simulation is based on one of the tutorials we provide in the [*JS-son* README](https://github.com/TimKam/JS-son#belief-desire-intention-plan-approach).
In this notebook, we simulate the spread of a single boolean belief among 100 agents in environments with different biases regarding the facilitation of the different opinion values.

Belief spread is simulated as follows:

* The scenario starts with each agent announcing their beliefs.

* In each iteration, the environment distributes two belief announcements to each agent. Based on these beliefs and possibly (depending on the agent type) the past announcements the agent was exposed to, each agent *announces* a new belief: either ``true`` or ``false``.

The agents are of two different agent types (``volatile`` and ``introspective``):

1. Type ``volatile``: Volatile agents only consider their current belief and the latest belief set they received from the environment when deciding which belief to announce. Volatile agents are "louder", i.e. the environment is more likely to spread beliefs of volatile agents. We also add bias to the announcement spread function to favor ``true`` announcements.

2. Type ``introspective``: In contrast to volatile agents, introspective agents consider the past five belief sets they have received, when deciding which belief they should announce. Introspective agents are "less loud", i.e. the environment is less likely to spread beliefs of volatile agents.

The agent type distribution is 50, 50.
However, 30 volatile and 20 introspective agents start with ``true`` as their belief, whereas 20 volatile and 30 introspective agents start with ``false`` as
their belief.


## Node.js - Python Interoperability
First, we install and import the ``pixiedust_node`` Python package that allows interoperability between Node.js and Python in Jupyter notebooks:

In [ ]:
%%capture
# We are getting a lot of output here that we do not want to see.
!pip install pixiedust_node
import pixiedust_node

## *JS-son* Scenario

First, we need to install the *JS-son* (``js-son-agent``) node module:

In [ ]:
npm.install('js-son-agent')

Then, we import the *JS-son* dependencies:

In [ ]:
%%capture
%%node
const {
  Belief,
  Desire,
  Intentions,
  Plan,
  Agent,
  Environment
} = require('js-son-agent')

We create the belief sets the agents start with:

In [ ]:
%%node
const beliefsTrue = {
  ...Belief('keyBelief', true),
  ...Belief('pastReceivedAnnouncements', [])
}

const beliefsFalse = {
  ...Belief('keyBelief', false),
  ...Belief('pastReceivedAnnouncements', [])
}

Now, we define the desires of the two agent types. Both agents base their announcement desires on
the predominant belief in previous announcements (see the ``determinePredominantBelief`` function).
However, volatile agents only consider the most recent round of announcements, while introspective
agents consider the whole history they have available. If both ``true`` and ``false`` occur equally
often in the considered announcement history, the currently held belief is considered to reach a
decision.

In [ ]:
%%node
const determinePredominantBelief = beliefs => {
  const announcementsTrue = beliefs.pastReceivedAnnouncements.filter(
    announcement => announcement
  ).length
  const announcementsFalse = beliefs.pastReceivedAnnouncements.filter(
    announcement => !announcement
  ).length
  const predominantBelief = announcementsTrue > announcementsFalse ||
    (announcementsTrue === announcementsFalse && beliefs.keyBelief)
  return predominantBelief
}

const desiresVolatile = {
  ...Desire('announceTrue', beliefs => {
    const pastReceivedAnnouncements = beliefs.pastReceivedAnnouncements.length >= 5
      ? beliefs.pastReceivedAnnouncements.slice(-5)
      : new Array(5).fill(beliefs.keyBelief)
    const recentBeliefs = {
      ...beliefs,
      pastReceivedAnnouncements
    }
    return determinePredominantBelief(recentBeliefs)
  }),
  ...Desire('announceFalse', beliefs => {
    const pastReceivedAnnouncements = beliefs.pastReceivedAnnouncements.length >= 5
      ? beliefs.pastReceivedAnnouncements.slice(-5)
      : new Array(5).fill(beliefs.keyBelief)
    const recentBeliefs = {
      ...beliefs,
      pastReceivedAnnouncements
    }
    return !determinePredominantBelief(recentBeliefs)
  })
}

const desiresIntrospective = {
  ...Desire('announceTrue', beliefs => determinePredominantBelief(beliefs)),
  ...Desire('announceFalse', beliefs => !determinePredominantBelief(beliefs))
}

The agents desires are mutually exclusive. Hence, the agents' intentions merely relay their desires,
which is reflected in the default preference function generator ``(beliefs, desires) => desireKey => desires[desireKey](beliefs)``.

The agents' plans are to disseminate the announcement (``true`` or ``false``) as determined by the
desire functions:

In [ ]:
%%node
const plans = [
  Plan(intentions => intentions.announceTrue, () => [ { announce: true } ]),
  Plan(intentions => intentions.announceFalse, () => [ { announce: false } ])
]

Before we instantiate the agents, we need to create an object for the environment's initial state.
The object will be populated when the agents will be created:

In [ ]:
%%node
const state = {}

To instantiate the agents according to the scenario specification, we create the following function:

In [ ]:
%%node
const createAgents = () => {
  const agents = new Array(100).fill({}).map((_, index) => {
    // assign agent types--introspective and volatile--to odd and even numbers, respectively:
    const type = index % 2 === 0 ? 'volatile' : 'introspective'
    const desires = type === 'volatile' ? desiresVolatile : desiresIntrospective
    /* ``true`` as belief: 30 volatile and 20 introspective agents
       ``false`` as belief: 20 volatile and 30 introspective agents:
    */
    const beliefs = (index < 50 && index % 2 === 0) || (index < 40 && index % 2 !== 0) ? beliefsTrue
      : beliefsFalse
    // add agent belief to the environment's state:
    state[`${type}${index}`] = { keyBelief: beliefs.keyBelief }
    // create agent:
    return new Agent(
      `${type}${index}`,
      { ...beliefs, ...Belief('type', type) },
      desires,
      plans
    )
  })
  const numberBeliefsTrue = Object.keys(state).filter(
    agentId => state[agentId].keyBelief
  ).length
  const numberBeliefsFalse = Object.keys(state).filter(
    agentId => !state[agentId].keyBelief
  ).length
  console.log(`True: ${numberBeliefsTrue}; False: ${numberBeliefsFalse}`)
  return agents
}

To define how the environment processes agent actions, we implement the ``updateState`` function.
The function takes an agent's actions, as well as the agent ID and the current state to determine
the environment's state update that is merged into the new state
``state = { ...state, ...stateUpdate }``:

In [ ]:
%%node
const updateState = (actions, agentId, currentState) => {
  const stateUpdate = {}
  actions.forEach(action => {
    stateUpdate[agentId] = {
      keyBelief: action.find(action => action.announce !== undefined).announce
    }
  })
  return stateUpdate
}

We simulate a partially observable world: via the environment's ``stateFilter`` function, we
determine an array of five belief announcements that should be made available to an agent. As
described in the specification, announcements of volatile agents will be "amplified": i.e., the
function pseudo-randomly picks 3 announcements of volatile agents and 2 announcements of
introspective agents. In addition, we add bias may facilitate either ``true`` of ``false``announcements: we implement a ``stateFilterGenerator`` function that can create state filters with different biases:

In [ ]:
%%node
const stateFilterGenerator = bias => (state, agentKey, agentBeliefs) => {
  const volatileAnnouncements = []
  const introspectiveAnnouncements = []
  Object.keys(state).forEach(key => {
    if (key.includes('volatile')) {
      volatileAnnouncements.push(state[key].keyBelief)
    } else {
      introspectiveAnnouncements.push(state[key].keyBelief)
    }
  })
  const recentVolatileAnnouncements = volatileAnnouncements.sort(
    () => 0.5 - Math.random()
  ).slice(0, 3)
  const recentIntrospectiveAnnouncements = introspectiveAnnouncements.sort(
    () => 0.5 - Math.random()
  ).slice(0, 2)
  // add some noise
  let noise = Object.keys(state).filter(agentId => state[agentId].keyBelief).length < (79 - bias) * Math.random() ? [true] : []
  noise = Object.keys(state).filter(agentId => state[agentId].keyBelief).length < bias * Math.random() ? [false] : noise
  // combine announcements
  const pastReceivedAnnouncements =
    recentVolatileAnnouncements.concat(
      recentIntrospectiveAnnouncements, agentBeliefs.pastReceivedAnnouncements, noise
    )
  return { pastReceivedAnnouncements, keyBelief: state[agentKey].keyBelief }
}

The last function we need is ``render()``. In our case, we simply log the number of announcements
of ``true`` and ``false`` to the console:

In [ ]:
%%node
const render = state => {
  const numberBeliefsTrue = Object.keys(state).filter(
    agentId => state[agentId].keyBelief
  ).length
  const numberBeliefsFalse = Object.keys(state).filter(
    agentId => !state[agentId].keyBelief
  ).length
  console.log(`True: ${numberBeliefsTrue}; False: ${numberBeliefsFalse}`)
}


We want to explore how different biases affect the simulation results (belief spread in the agent society). We generate 20 state filters with different biases:

In [ ]:
%%node
const stateFilters = new Array(20).fill(0).map((_, index) => stateFilterGenerator(20  + index))

We instantiate the environments with the specified agents, state, update function, render
function, and **different** ``stateFilter`` functions:

In [ ]:
%%node
var histories = stateFilters.map(stateFilter => new Environment(
  createAgents(),
  state,
  updateState,
  render,
  stateFilter
).run(20));

## Analytics
To analyze the simulation data, we switch to Python tools.
We import a set of data management and visualization libraries:

In [ ]:
%%capture
import pandas as pd
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt
import seaborn as sns
import time
while not 'histories' in globals():
  time.sleep(1); # Unfortunately, we have wait for Node.js and Python to sync


Next, we parse the data from Python dictionaries (as provided by the Node.js - Python interface) to a dictionary, that can later be turned into a ``pandas`` dataframes:

In [ ]:
belief_summaries = []
for history in histories:
    beliefs_true_history = []
    beliefs_false_history = []
    for envState in enumerate(history):
        beliefs_true_count = 0
        beliefs_false_count = 0
        for key in envState[1].keys():
            if envState[1][key]['keyBelief']:
                beliefs_true_count += 1
            else:
                beliefs_false_count +=1     
        beliefs_true_history.append(beliefs_true_count)
        beliefs_false_history.append(beliefs_false_count)

    belief_summary = {
        'true': beliefs_true_history,
        'false': beliefs_false_history,
    }
    belief_summaries.append(belief_summary);
    

Finally, we instantiate an interactive widget that allows comparing time series charts with different environment biases:

In [ ]:
def f(bias_1, bias_2):
    belief_summary_df_1 = pd.DataFrame(belief_summaries[bias_1])
    label_1 = 'true (bias: ' + str(bias_1) + ')'
    label_2 = 'false (bias: ' + str(bias_1) + ')'
    label_3 = 'true (bias: ' + str(bias_2) + ')'
    label_4 = 'false (bias: ' + str(bias_2) + ')'
    belief_summary_df_1.columns = [label_1, label_2]
    belief_summary_df_2 = pd.DataFrame(belief_summaries[bias_2])
    belief_summary_df_2.columns = [label_3, label_4]
    sns.lineplot(data=[belief_summary_df_1[label_1], belief_summary_df_1[label_2],
                       belief_summary_df_2[label_3], belief_summary_df_2[label_4]])
    
interact(
    f,
    bias_1=widgets.IntSlider(min=0,max=19,step=1,value=0),
    bias_2=widgets.IntSlider(min=0,max=19,step=1,value=19)
);